# Imports

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
sns.set(rc={'figure.figsize':(25,10)})


# Transform file to csv

In [123]:
# This is a function definition in Python that takes two arguments: `filename` and `new_filename`, both of which are strings. 
# The function reads the content of an Excel file using the pandas library's `read_excel` function and stores it in a dataframe object. 
# It then writes the contents of the dataframe object to a CSV file using the `to_csv` function,
# with the `index` parameter set to `None` and `header` parameter set to `True`. 
# The function returns `None`. This function can be used to convert an Excel file to a CSV file.

def transform_xls_to_csv(filename: str, new_filename: str) -> None:
    # Read and store content of an excel file 
    read_file = pd.read_excel (filename)
    # Write the dataframe object into csv file
    read_file.to_csv (new_filename, index = None, header=True)

In [124]:
#transform_xls_to_csv("../raw_data/bookings_without_onepark.xlsx","../raw_data/bookings_without_onepark.csv")

In [125]:
# This code is defining a dictionary called `custom_dtype` that maps column names to their respective data types.
# The keys of the dictionary are the column names and the values are the data types. 
# The data types include `str` for string, and `float` for floating-point numbers. 
# This dictionary can be used to specify the data types of columns when reading data from a file or database.

custom_dtype = {'id': str,                        
                'pocket':str,
                'product':str,
                'status':str,
                'option':str,
                'guest_id':str,
                'booking_fees':float,
                'amount':float,
                'total_amount':float,
                'discount':float,
                'creation_date_hour':str,
                'beginning_date_hour':str,
                'begining_slice':str,
                'end_date_hour':str,
                'max_date_hour':str,
                'cxl_date_hour':str,
                'los':str,
                'lead_time_hours':float
                }

In [126]:
# This code reads a CSV file named "bookings_without_onepark.csv" located in the "../raw_data/" 
# directory and stores the data in a pandas DataFrame called "df". The "dtype" parameter specifies
# the data types of the columns in the DataFrame, which are defined in a custom dictionary 
# called "custom_dtype". The "parse_dates" parameter specifies which columns should be parsed as datetime objects.
# In this case, the columns "creation_date_hour", "beginning_date_hour", and "end_date_hour" are being parsed as datetime objects.


custom_dtype = {
    'id': int
}

df = pd.read_csv("../raw_data/bookings_without_onepark.csv", 
                 dtype=custom_dtype, 
                 parse_dates=['creation_date_hour', 'beginning_date_hour', 'end_date_hour'])





/var/folders/gn/32bg35111_q711w0ydw7_2rc0000gp/T/ipykernel_45383/1038938611.py:12: DtypeWarning:

Columns (1,14,18,19) have mixed types. Specify dtype option on import or set low_memory=False.



# Cleaning the DF 

In [127]:
# Drop the 'entry_date_hour' and 'exit_date_hour' columns
df = df.drop(['entry_date_hour', 'exit_date_hour','promo_code','amount_promo'], axis=1)

In [128]:
# Check for duplicates based on the 'id' column
duplicates = df.duplicated(subset=['id'])

# Count the number of duplicates
duplicate_count = duplicates.sum()

# Print the number of duplicates
print("Number of duplicates:", duplicate_count)

# Drop the duplicates based on the 'id' column
df = df.drop_duplicates(subset=['id'])

# Print the updated DataFrame without duplicates
print(df.head())

Number of duplicates: 1035
        id  pocket product    status    option    guest_id  booking_fees  \
0  5144307      36     H10  finished  standard  DCL0300508           1.0   
1  5144312     781     H10  finished  standard  DCL0266680           3.0   
2  5144315     491     H10  finished  standard  DCL0286500           3.5   
3  5144319  600287     H10  finished  standard  DCL0254021           4.0   
4  5144320  600287     H10  canceled  standard  DCL0271067           4.0   

   amount  total_amount  discount      creation_date_hour beginning_date_hour  \
0    58.2         59.20      0.00 2019-01-01 09:11:01.000 2019-02-07 15:00:00   
1    29.5         32.50      0.00 2019-01-01 09:32:29.000 2019-01-08 07:30:00   
2    28.8         23.04      9.26 2019-01-01 09:45:16.999 2019-01-02 06:30:00   
3    40.0         44.00      0.00 2019-01-01 09:58:58.999 2019-01-31 05:45:00   
4    56.0         60.00      0.00 2019-01-01 09:59:13.000 2019-01-17 12:30:00   

  begining_slice           en

In [129]:
# Check for nan values and transform them to unknown to count them and to 0 after
df = df.fillna('unknown')
df.isin(['unknown']).sum()
df.replace({'unknown': 0})

,id,pocket,product,status,option,guest_id,booking_fees,amount,total_amount,discount,creation_date_hour,beginning_date_hour,begining_slice,end_date_hour,max_date_hour,cxl_date_hour,los,lead_time_hours
0,5144307,36,H10,finished,standard,DCL0300508,1.0,58.2,59.20,0.00,2019-01-01 09:11:01.000,2019-02-07 15:00:00.000,15H à 18H,2019-02-10 19:44:59.999,0,0,+24h,893.816389
1,5144312,781,H10,finished,standard,DCL0266680,3.0,29.5,32.50,0.00,2019-01-01 09:32:29.000,2019-01-08 07:30:00.000,6H à 9H,2019-01-10 21:30:00.000,0,0,+24h,165.958611
2,5144315,491,H10,finished,standard,DCL0286500,3.5,28.8,23.04,9.26,2019-01-01 09:45:16.999,2019-01-02 06:30:00.000,6H à 9H,2019-01-04 21:45:00.000,0,0,+24h,20.745278
3,5144319,600287,H10,finished,standard,DCL0254021,4.0,40.0,44.00,0.00,2019-01-01 09:58:58.999,2019-01-31 05:45:00.000,0H à 6H,2019-02-02 17:45:00.000,0,0,+24h,715.766944
4,5144320,600287,H10,canceled,standard,DCL0271067,4.0,56.0,60.00,0.00,2019-01-01 09:59:13.000,2019-01-17 12:30:00.000,12H à 15H,2019-01-20 15:59:59.999,0,2019-01-11 18:26:00.000,+24h,386.513056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536169,10276847,0000500287,F397,completed,standard,DCL0129618,0.0,46.5,46.50,0.00,2023-06-18 23:17:23.999,2023-06-23 07:44:59.999,6H à 9H,2023-06-26 17:15:00.000,2023-06-27 07:44:59.999,0,+24h,104.460000
536170,10276859,0000000781,F398,completed,standard,DCL0779147,0.0,46.0,46.00,0.00,2023-06-18 23:22:06.000,2023-07-04 05:29:59.999,0H à 6H,2023-07-10 05:29:59.999,2023-07-11 05:29:59.999,0,+24h,366.131667
536171,10276871,0000000206,F397,completed,standard,DCL0524194,0.0,29.9,29.90,0.00,2023-06-18 23:30:14.000,2023-06-23 15:59:59.999,15H à 18H,2023-06-25 23:29:59.999,2023-06-26 11:59:00.000,0,+24h,112.496111
536172,10276885,0000000206,H10,completed,standard,DCL0311148,0.0,56.9,56.90,0.00,2023-06-18 23:36:58.000,2023-06-19 14:45:00.000,12H à 15H,2023-06-21 22:44:59.999,2023-06-22 02:45:00.000,0,+24h,15.133889


In [130]:
# remove electrique option
df = df.drop(df[df['option'] == 'electrique'].index)
df

,id,pocket,product,status,option,guest_id,booking_fees,amount,total_amount,discount,creation_date_hour,beginning_date_hour,begining_slice,end_date_hour,max_date_hour,cxl_date_hour,los,lead_time_hours
0,5144307,36,H10,finished,standard,DCL0300508,1.0,58.2,59.20,0.00,2019-01-01 09:11:01.000,2019-02-07 15:00:00.000,15H à 18H,2019-02-10 19:44:59.999,unknown,unknown,+24h,893.816389
1,5144312,781,H10,finished,standard,DCL0266680,3.0,29.5,32.50,0.00,2019-01-01 09:32:29.000,2019-01-08 07:30:00.000,6H à 9H,2019-01-10 21:30:00.000,unknown,unknown,+24h,165.958611
2,5144315,491,H10,finished,standard,DCL0286500,3.5,28.8,23.04,9.26,2019-01-01 09:45:16.999,2019-01-02 06:30:00.000,6H à 9H,2019-01-04 21:45:00.000,unknown,unknown,+24h,20.745278
3,5144319,600287,H10,finished,standard,DCL0254021,4.0,40.0,44.00,0.00,2019-01-01 09:58:58.999,2019-01-31 05:45:00.000,0H à 6H,2019-02-02 17:45:00.000,unknown,unknown,+24h,715.766944
4,5144320,600287,H10,canceled,standard,DCL0271067,4.0,56.0,60.00,0.00,2019-01-01 09:59:13.000,2019-01-17 12:30:00.000,12H à 15H,2019-01-20 15:59:59.999,unknown,2019-01-11 18:26:00.000,+24h,386.513056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536169,10276847,0000500287,F397,completed,standard,DCL0129618,0.0,46.5,46.50,0.00,2023-06-18 23:17:23.999,2023-06-23 07:44:59.999,6H à 9H,2023-06-26 17:15:00.000,2023-06-27 07:44:59.999,unknown,+24h,104.460000
536170,10276859,0000000781,F398,completed,standard,DCL0779147,0.0,46.0,46.00,0.00,2023-06-18 23:22:06.000,2023-07-04 05:29:59.999,0H à 6H,2023-07-10 05:29:59.999,2023-07-11 05:29:59.999,unknown,+24h,366.131667
536171,10276871,0000000206,F397,completed,standard,DCL0524194,0.0,29.9,29.90,0.00,2023-06-18 23:30:14.000,2023-06-23 15:59:59.999,15H à 18H,2023-06-25 23:29:59.999,2023-06-26 11:59:00.000,unknown,+24h,112.496111
536172,10276885,0000000206,H10,completed,standard,DCL0311148,0.0,56.9,56.90,0.00,2023-06-18 23:36:58.000,2023-06-19 14:45:00.000,12H à 15H,2023-06-21 22:44:59.999,2023-06-22 02:45:00.000,unknown,+24h,15.133889


In [131]:
# Count the occurrences of each product and store it in a new DataFrame
product_counts = df['product'].value_counts().reset_index()

# Rename the columns in the new DataFrame
product_counts.columns = ['product', 'count']

# Print the product names and their corresponding counts
print(product_counts)

   product   count
0      H10  313265
1     F397   73963
2     F398   50106
3      F44   40795
4      F98   35485
5      F60    5078
6      F63    4946
7     F109    2996
8     F132    1573
9     F139    1117
10    F400     999
11    F414     995
12    F150     680
13     F54     632
14     F67     495
15    F343     451
16    F138     421
17      F7     378
18     F33     231
19    F319     229
20     F70     106
21    F100      77
22    F110      33
23    F227      20
24    F137      20
25     F58      18
26      F5       1
27    F215       1


In [132]:
# Count the occurrences of each pocket and store it in a new DataFrame
pocket_counts = df['pocket'].value_counts().reset_index()

# Rename the columns in the new DataFrame
pocket_counts.columns = ['pocket', 'count']

# Print the product names and their corresponding counts
print(pocket_counts.head(10))

       pocket  count
0  0000000781  81621
1  0000200287  60483
2  0000100435  32624
3  0000000206  27553
4  0000000272  21492
5  0000000036  19940
6  0000500435  18971
7  0000000034  18080
8  0000000052  15502
9  0000000025  15153


In [133]:
# Count the occurrences of each option
options = df['option'].value_counts().reset_index()
options

,index,option
0,standard,493057
1,premium,42054


In [134]:
# Replace the names of the products
df['product'] = df['product'].replace({'H10': 'hourly rate', 
                                       'F397':'WE package',
                                       'F398': '1 week package',
                                       'F44': '1 week package',
                                       'F98':'WE package',
                                       'F60': '1 month package',
                                       'F63' : '2 weeks package',
                                       'F109': 'other package',
                                       'F132': 'other package',
                                       'F139': '1 month package',
                                       'F400':'1 month package',
                                       'F414': '2 weeks package',
                                       'F150': 'other package',
                                       'F54': '1 week package',
                                       'F67': 'other package',
                                       'F343': 'WE package',
                                       'F138': '1 week package',
                                       'F7': 'other package',
                                       'F33': '2 weeks package',
                                       'F319':'1 week package',
                                       'F70': '1 month package',
                                       'F100': 'other package',
                                       'F110': 'other package',
                                       'F227': 'other package',
                                       'F137': 'other package',
                                       'F58': 'other package',
                                       'F5' : '1 month package',
                                       'F215': 'other package'
                                       })

# Print the updated DataFrame
print(df)

              id      pocket         product     status    option    guest_id  \
0        5144307          36     hourly rate   finished  standard  DCL0300508   
1        5144312         781     hourly rate   finished  standard  DCL0266680   
2        5144315         491     hourly rate   finished  standard  DCL0286500   
3        5144319      600287     hourly rate   finished  standard  DCL0254021   
4        5144320      600287     hourly rate   canceled  standard  DCL0271067   
...          ...         ...             ...        ...       ...         ...   
536169  10276847  0000500287      WE package  completed  standard  DCL0129618   
536170  10276859  0000000781  1 week package  completed  standard  DCL0779147   
536171  10276871  0000000206      WE package  completed  standard  DCL0524194   
536172  10276885  0000000206     hourly rate  completed  standard  DCL0311148   
536173  10276914  0000000036   other package  completed  standard  DCL0669147   

        booking_fees  amoun

In [135]:
# Count the occurrences of each product and store it in a new DataFrame
product_counts = df['product'].value_counts().reset_index()

# Rename the columns in the new DataFrame
product_counts.columns = ['product', 'count']

# Print the product names and their corresponding counts
print(product_counts)

           product   count
0      hourly rate  313265
1       WE package  109899
2   1 week package   92183
3  1 month package    7301
4    other package    6291
5  2 weeks package    6172


# Creation of df_days, with the nb of bookings, cars and cancellations per days

In [136]:
# Creation of a DataFrame with 1 car for each day, cancellations (cxl), 
# and nb of bookings and cancellations by days of arrival
df_days = pd.DataFrame({
    'date': pd.date_range(start='2021-06-01', end='2023-06-20'),
    'nb_cars': 0,
    'nb_cars_cxl': 0,
    'nb_bookings': 0,
    'nb_bookings_cxl': 0
})

# Print the updated DataFrame
print(df_days.head())


        date  nb_cars  nb_cars_cxl  nb_bookings  nb_bookings_cxl
0 2021-06-01        0            0            0                0
1 2021-06-02        0            0            0                0
2 2021-06-03        0            0            0                0
3 2021-06-04        0            0            0                0
4 2021-06-05        0            0            0                0


In [137]:
# Iterate through each row of the DataFrame `df`
for _, row in df.iterrows():
    start = row['beginning_date_hour']
    end = row['end_date_hour']
    status = row['status']
    
    # Update nb_cars column
    df_days.loc[(df_days['date'] >= start) & (df_days['date'] <= end) & (status != 'canceled'), 'nb_cars'] += 1
    
    # Update nb_cars_canceled column
    df_days.loc[(df_days['date'] >= start) & (df_days['date'] <= end) & (status == 'canceled'), 'nb_cars_cxl'] += 1



In [138]:
# Count the number of bookings and cancellations for each date
booking_counts = df.loc[df['status'] != 'canceled', 'beginning_date_hour'].dt.date.value_counts()
cancellation_counts = df.loc[df['status'] == 'canceled', 'beginning_date_hour'].dt.date.value_counts()

# Update nb_bookings column
df_days['nb_bookings'] = df_days['date'].map(booking_counts).fillna(0).astype(int)

# Update nb_bookings_cxl column
df_days['nb_bookings_cxl'] = df_days['date'].map(cancellation_counts).fillna(0).astype(int)


In [139]:
df_days.head()

,date,nb_cars,nb_cars_cxl,nb_bookings,nb_bookings_cxl
0,2021-06-01,368,30,172,12
1,2021-06-02,395,34,169,4
2,2021-06-03,428,26,170,13
3,2021-06-04,412,25,424,30
4,2021-06-05,695,49,187,16


In [140]:
# Save the df to csv 
df_days.to_csv("../prepared_data/old_df/01_06_2021_to_20_06_2023_anais.csv", index=False)


In [141]:
# Convert 'date' column to datetime data type
df_days['date'] = pd.to_datetime(df_days['date'])

# Reshape the DataFrame using melt
df_melted = df_days.melt(id_vars='date', value_vars=['nb_cars', 'nb_cars_cxl', 'nb_bookings', 'nb_bookings_cxl'])

# Create the line plot
fig = go.Figure()

# Add traces for each variable
for variable in ['nb_cars', 'nb_cars_cxl', 'nb_bookings', 'nb_bookings_cxl']:
    fig.add_trace(go.Scatter(
        x=df_melted['date'],
        y=df_melted[df_melted['variable'] == variable]['value'],
        name=variable
    ))

# Add range slider and selectors
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True)
    ),
    title="Number of cars and bookings per day"
)

# Set the height of the plot
fig.update_layout(height=800)

# Show the graph
fig.show()


# Creation of df_products with the nb of cars per days by products (w/o cancellations)

In [142]:
# Creation of a DataFrame with 1 car for each day without cancellations, displayed by products 

df_products = pd.DataFrame({
    'date': pd.date_range(start='2021-06-01', end='2023-06-20'),
    'hourly rate': 0,
    'WE package': 0,
    '1 week package': 0,
    '1 month package':0,
    'other package': 0,
    '2 weeks package': 0,   
})

# Print the updated DataFrame
print(df_products.head())

        date  hourly rate  WE package  1 week package  1 month package  \
0 2021-06-01            0           0               0                0   
1 2021-06-02            0           0               0                0   
2 2021-06-03            0           0               0                0   
3 2021-06-04            0           0               0                0   
4 2021-06-05            0           0               0                0   

   other package  2 weeks package  
0              0                0  
1              0                0  
2              0                0  
3              0                0  
4              0                0  


In [143]:
# Iterate through each row of the DataFrame `df`
for _, row in df.iterrows():
    start = row['beginning_date_hour']
    end = row['end_date_hour']
    status = row['status']
    product = row['product']  # Retrieve the product value from the row
    
    # Update the count for the respective product
    df_products.loc[(df_products['date'] >= start) & (df_products['date'] <= end) & (status != 'canceled'), product] += 1


In [144]:
df_products.head()

,date,hourly rate,WE package,1 week package,1 month package,other package,2 weeks package
0,2021-06-01,115,14,176,53,5,5
1,2021-06-02,160,1,171,55,3,5
2,2021-06-03,178,0,189,52,4,5
3,2021-06-04,150,0,205,48,3,6
4,2021-06-05,204,236,199,45,3,8


In [145]:
# Save the df to csv 
df_products.to_csv("../prepared_data/parts_of_final_df/01_06_2021_to_20_06_2023_products_anais.csv", index=False)

In [146]:
# Plot a graph of the number of cars per day and products
wide_df = pd.DataFrame(df_products)

fig = px.bar(wide_df, 
             x="date", 
             y=["hourly rate", "WE package", "1 week package", "1 month package", "other package", "2 weeks package"], 
             title="Number of cars per day & product")

# Add a range slider and selectors
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Set the height of the plot
fig.update_layout(height=800)

# Show the graph
fig.show()


# Creation of df_turnover

In [162]:
# Initialize the DataFrame with the date column
df_turnover = pd.DataFrame({
    'date': pd.date_range(start='2021-06-01', end='2023-06-20')
})

# Calculate turnover by date
df['date'] = pd.to_datetime(df['beginning_date_hour']).dt.date  # Convert 'beginning_date_hour' to date
turnover_data = df.loc[df['status'] != 'canceled'].groupby('date').agg({'amount': 'sum'}).reset_index()
turnover_data = turnover_data.rename(columns={'amount': 'turnover'})
turnover_data['date'] = pd.to_datetime(turnover_data['date'])  # Convert 'date' to datetime

# Calculate discount by date
discount_data = df.loc[df['status'] != 'canceled'].groupby('date').agg({'discount': 'sum'}).reset_index()
discount_data['date'] = pd.to_datetime(discount_data['date'])  # Convert 'date' to datetime

# Calculate booking fees by date
booking_fees_data = df.loc[df['status'] != 'canceled'].groupby('date').agg({'booking_fees': 'sum'}).reset_index()
booking_fees_data['date'] = pd.to_datetime(booking_fees_data['date'])  # Convert 'date' to datetime

# Calculate mean of lead_time_hours by date
mean_lead_time_data = df.groupby('date')['lead_time_hours'].mean().reset_index()
mean_lead_time_data['date'] = pd.to_datetime(mean_lead_time_data['date'])  # Convert 'date' to datetime

# Merge the data into the df_turnover DataFrame
df_turnover = df_turnover.merge(turnover_data, on='date', how='left')
df_turnover = df_turnover.merge(discount_data, on='date', how='left')
df_turnover = df_turnover.merge(booking_fees_data, on='date', how='left')
df_turnover = df_turnover.merge(mean_lead_time_data, on='date', how='left')

# Fill NaN values with 0
df_turnover[['turnover', 'discount', 'booking_fees', 'lead_time_hours']] = df_turnover[['turnover', 'discount', 'booking_fees', 'lead_time_hours']].fillna(0)

# Print the updated DataFrame
print(df_turnover.head())


        date  turnover  discount  booking_fees  lead_time_hours
0 2021-06-01    6672.1    171.00          25.1       100.263407
1 2021-06-02    5601.9    140.04          36.1       154.039192
2 2021-06-03    6409.8    207.81          51.3       124.986492
3 2021-06-04   14215.1   1048.70         139.9       151.524811
4 2021-06-05    5812.1    355.23          53.0       170.620814


In [163]:
# Save the df to csv 
df_turnover.to_csv("../prepared_data/parts_of_final_df/01_06_2021_to_20_06_2023_turnover_anais.csv", index=False)

In [174]:
# Multiply the discount column by -1 to make it negative
df_turnover['discount'] = df_turnover['discount'] * -1

# Reshape the DataFrame using melt
df_melted = df_turnover[['date', 'turnover', 'booking_fees', 'discount']].melt(id_vars='date', var_name='category', value_name='amount')

# Create the bar chart
fig = px.bar(df_melted, x='date', y='amount', color='category', barmode='relative',
             labels={'amount': 'Amount', 'category': 'Category'})

# Set the y-axis range to include negative values
fig.update_layout(yaxis_range=[df_melted['amount'].min(), df_melted['amount'].max()])

# Customize the layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Amount',
    legend_title='Category',
    barmode='relative',
    bargap=0.2,
    bargroupgap=0.1
)

# Add range slider and selectors
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True)
    ),
    title="Turnover by days (amount of booking, booking fees, and discount)"
)

# Set the height of the plot
fig.update_layout(height=800)

# Show the graph
fig.show()



# Adding the last columns of the original dataset

In [178]:
# Retrieve the values of the los
los_counts

,index,los
0,+24h,448339
1,06:00 24:00,78228
2,00:30 06:00,8290
3,00:00 00:30,254


In [177]:
# Creation of a DataFrame with 1 car for each day without cancellations, displayed by products 

df_option = pd.DataFrame({
    'date': pd.date_range(start='2021-06-01', end='2023-06-20'),
    'standard': 0,
    'premium': 0,
    '6H à 9H': 0,
    '15H à 18H': 0,
    '9H à 12H': 0,
    '12H à 15H': 0,
    '0H à 6H': 0,
    '18H à 24H': 0,
    '+24h': 0,
    '06:00 24:00': 0,
    '00:30 06:00': 0,
    '00:00 00:30': 0
})

# Iterate through each row of the DataFrame `df`
for _, row in df.iterrows():
    start = row['beginning_date_hour']
    end = row['end_date_hour']
    status = row['status']
    option = row['option']  # Retrieve the option value from the row
    begining_slice = row['begining_slice']  # Retrieve the option value from the row
    los = row['los']  # Retrieve the option value from the row
    
    # Update the count for the respective option
    df_option.loc[(df_option['date'] >= start) & (df_option['date'] <= end) & (status != 'canceled'), option] += 1
    df_option.loc[(df_option['date'] >= start) & (df_option['date'] <= end) & (status != 'canceled'), begining_slice] += 1    
    df_option.loc[(df_option['date'] >= start) & (df_option['date'] <= end) & (status != 'canceled'), los] += 1   
    

print(df_option.head())

        date  standard  premium  6H à 9H  15H à 18H  9H à 12H  12H à 15H  \
0 2021-06-01       351       17      112         67        90         59   
1 2021-06-02       381       14      122         67        96         68   
2 2021-06-03       414       14      131         75       105         72   
3 2021-06-04       402       10      119         83        94         67   
4 2021-06-05       680       15      134        220       117        133   

   0H à 6H  18H à 24H  +24h  06:00 24:00  00:30 06:00  00:00 00:30  
0       22         18   365            3            0            0  
1       24         18   395            0            0            0  
2       26         19   428            0            0            0  
3       28         21   411            1            0            0  
4       28         63   694            1            0            0  


In [179]:
# Save the df to csv 
df_option.to_csv("../prepared_data/parts_of_final_df/01_06_2021_to_20_06_2023_option_anais.csv", index=False)

# Import DF

In [196]:
# Load prepared dataset
df_days = pd.read_csv("../prepared_data/old_df/01_06_2021_to_20_06_2023_anais.csv")
print(df_days.head())


         date  nb_cars  nb_cars_cxl  nb_bookings  nb_bookings_cxl
0  2021-06-01      368           30          172               12
1  2021-06-02      395           34          169                4
2  2021-06-03      428           26          170               13
3  2021-06-04      412           25          424               30
4  2021-06-05      695           49          187               16


In [197]:
# Load products dataset
df_products = pd.read_csv("../prepared_data/parts_of_final_df/01_06_2021_to_20_06_2023_products_anais.csv")
print(df_products.head())


         date  hourly rate  WE package  1 week package  1 month package  \
0  2021-06-01          115          14             176               53   
1  2021-06-02          160           1             171               55   
2  2021-06-03          178           0             189               52   
3  2021-06-04          150           0             205               48   
4  2021-06-05          204         236             199               45   

   other package  2 weeks package  
0              5                5  
1              3                5  
2              4                5  
3              3                6  
4              3                8  


In [198]:
# Load turnover dataset
df_turnover = pd.read_csv("../prepared_data/parts_of_final_df/01_06_2021_to_20_06_2023_turnover_anais.csv")
print(df_turnover.head())

         date  turnover  discount  booking_fees  lead_time_hours
0  2021-06-01    6672.1    171.00          25.1       100.263407
1  2021-06-02    5601.9    140.04          36.1       154.039192
2  2021-06-03    6409.8    207.81          51.3       124.986492
3  2021-06-04   14215.1   1048.70         139.9       151.524811
4  2021-06-05    5812.1    355.23          53.0       170.620814


In [199]:
# Load option dataset
df_option = pd.read_csv("../prepared_data/parts_of_final_df/01_06_2021_to_20_06_2023_option_anais.csv")
print(df_option.head())

         date  standard  premium  6H à 9H  15H à 18H  9H à 12H  12H à 15H  \
0  2021-06-01       351       17      112         67        90         59   
1  2021-06-02       381       14      122         67        96         68   
2  2021-06-03       414       14      131         75       105         72   
3  2021-06-04       402       10      119         83        94         67   
4  2021-06-05       680       15      134        220       117        133   

   0H à 6H  18H à 24H  +24h  06:00 24:00  00:30 06:00  00:00 00:30  
0       22         18   365            3            0            0  
1       24         18   395            0            0            0  
2       26         19   428            0            0            0  
3       28         21   411            1            0            0  
4       28         63   694            1            0            0  


# Merging DF

In [204]:
# Merging DataFrames
new_df = df_days.merge(df_products, left_index=True, right_index=True, suffixes=('_days', '_products'))
new_df = new_df.merge(df_turnover, left_index=True, right_index=True, suffixes=('_merged', '_turnover'))
new_df = new_df.merge(df_option, left_index=True, right_index=True, suffixes=('_merged', '_option'))

new_df = new_df.drop(['date_products', 'date_merged', 'date_option' ], axis=1)
new_df = new_df.rename(columns={'date_days': 'date'})

new_df

,date,nb_cars,nb_cars_cxl,nb_bookings,nb_bookings_cxl,hourly rate,WE package,1 week package,1 month package,other package,...,6H à 9H,15H à 18H,9H à 12H,12H à 15H,0H à 6H,18H à 24H,+24h,06:00 24:00,00:30 06:00,00:00 00:30
0,2021-06-01,368,30,172,12,115,14,176,53,5,...,112,67,90,59,22,18,365,3,0,0
1,2021-06-02,395,34,169,4,160,1,171,55,3,...,122,67,96,68,24,18,395,0,0,0
2,2021-06-03,428,26,170,13,178,0,189,52,4,...,131,75,105,72,26,19,428,0,0,0
3,2021-06-04,412,25,424,30,150,0,205,48,3,...,119,83,94,67,28,21,411,1,0,0
4,2021-06-05,695,49,187,16,204,236,199,45,3,...,134,220,117,133,28,63,694,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2023-06-16,1450,99,1070,58,571,239,435,178,0,...,425,262,311,267,90,95,1431,19,0,0
746,2023-06-17,1858,112,583,37,261,1018,378,175,0,...,414,474,315,371,81,203,1853,5,0,0
747,2023-06-18,2119,121,268,12,337,1199,385,172,2,...,579,483,381,376,97,203,2078,40,1,0
748,2023-06-19,1299,84,846,59,202,480,422,174,0,...,381,242,279,244,54,99,1291,8,0,0


In [205]:
column_names = new_df.columns.tolist()
print(column_names)


['date', 'nb_cars', 'nb_cars_cxl', 'nb_bookings', 'nb_bookings_cxl', 'hourly rate', 'WE package', '1 week package', '1 month package', 'other package', '2 weeks package', 'turnover', 'discount', 'booking_fees', 'lead_time_hours', 'standard', 'premium', '6H à 9H', '15H à 18H', '9H à 12H', '12H à 15H', '0H à 6H', '18H à 24H', '+24h', '06:00 24:00', '00:30 06:00', '00:00 00:30']


In [206]:
# Save the df to csv 
new_df.to_csv("../prepared_data/01_06_2021_to_20_06_2023_prepared_df.csv", index=False)

# Adding strikes, holidays... to prepared_df

In [7]:
# Load strike dataset
df_strikes = pd.read_csv("../prepared_data/parts_of_final_df/strikes.csv")
print(df_strikes.tail())

          Date;strike
5508  07/07/2016;True
5509  08/07/2016;True
5510  09/07/2016;True
5511  10/07/2016;True
5512  11/07/2016;True
